In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns


import sklearn

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn import tree

In [2]:
class LoadData():
    def __init__(self,train_data_file):
        self.data = pd.read_csv(train_data_file)        
        

In [3]:
class Preprocessing():
    
    def pre_processing(self,dataset):
        dataset['LoanAmount'].fillna(dataset['LoanAmount'].median(), inplace = True)
        dataset['Self_Employed'].fillna('No', inplace=True)
        dataset['Credit_History'].fillna(2, inplace=True)
        dataset['Gender'].fillna('Male', inplace=True)
        dataset['Married'].fillna('Yes', inplace=True)
        #print(dataset.isnull().sum())
        dataset['Gender'] = dataset['Gender'].map({'Male':1,'Female':0})
        dataset['Married'] = dataset['Married'].map({'Yes':1,'No':0})
        dataset['Education'] = dataset['Education'].map({'Graduate':1,'Not Graduate':0})
        dataset['Self_Employed'] = dataset['Self_Employed'].map({'Yes':1,'No':0})
        dataset['Property_Area'] = dataset['Property_Area'].map({'Urban':1,'Rural':0,'Semiurban':2})
        dataset['Dependents'] = dataset['Dependents'].map({'0':1,'1':0,'2':2,'3+':3})
        print(dataset.isnull().sum())
        return dataset
    
    def data_analysis(self):
        sns.barplot(x = 'Gender', y = 'Loan_Status', data=self.train)
        sns.barplot(x = 'ApplicantIncome', y = 'Loan_Status',data=self.train)
        sns.barplot(x = 'Self_Employed', y = 'Loan_Status', data=self.train)
        #plt.scatter(self.x_train, self.y_train)
        #plt.show()
        
    def get_preprocess_data(self,data):
        dataset = self.pre_processing(data)
        #dataset['Loan_Status'] = self.label_encoder.fit_transform(dataset['Loan_Status'])
        dataset['Loan_Status'] = dataset['Loan_Status'].map({'Y':1,'N':0})
        print(dataset.head())
        print(len(dataset))
        dataset = dataset.dropna(axis=0)
        dataset = dataset.drop('Loan_ID',axis=1)
        print(dataset.isnull().sum())
        print(len(dataset))
        self.train,self.test = train_test_split(dataset,test_size = 0.1,random_state = 0,shuffle=False)
        
        self.y_train = self.train['Loan_Status']
        self.x_train = self.train.drop('Loan_Status',axis=1)
        
        
        self.y_test = self.test['Loan_Status']
        self.x_test = self.test.drop('Loan_Status',axis=1)
        #print(dataset.describe())

In [4]:
train_data = os.path.join('Data','train.csv')
load_data_obj = LoadData(train_data)

In [5]:
preprocess_obj = Preprocessing()
preprocess_obj.get_preprocess_data(load_data_obj.data)

#preprocess_obj.data_analysis()

Loan_ID               0
Gender                0
Married               0
Dependents           15
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term     14
Credit_History        0
Property_Area         0
Loan_Status           0
dtype: int64
    Loan_ID  Gender  Married  Dependents  Education  Self_Employed  \
0  LP001002       1        0         1.0          1              0   
1  LP001003       1        1         0.0          1              0   
2  LP001005       1        1         1.0          1              1   
3  LP001006       1        1         1.0          0              0   
4  LP001008       1        0         1.0          1              0   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0       128.0             360.0   
1             4583             1508.0       128.0             360.0   
2             3000                0.0    

In [6]:
class GenModel():
    def __init__(self):
        self.model = None
        self.x_train = preprocess_obj.x_train
        self.y_train = preprocess_obj.y_train
        self.x_test = preprocess_obj.x_test
        self.y_test = preprocess_obj.y_test
        
    def train(self):
        #self.model = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
        self.model=LogisticRegression(class_weight='balanced')

        self.model.fit(self.x_train,self.y_train)
        print(self.model.score(self.x_test,self.y_test))

In [7]:
model_obj = GenModel()
model_obj.train()

0.6440677966101694


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [13]:
test_data = pd.read_csv(os.path.join('Data','test.csv'),index_col='Loan_ID') 

In [16]:
class Prediction():
    def __init__(self):
        self.model = model_obj.model
    def prediction(self,test_data):
        df_test = preprocess_obj.pre_processing(test_data)
        y_predict=self.model.predict(df_test)
        y_predict=pd.DataFrame(y_predict,columns =['Loan_Status'],index=df_test.index)
        y_predict['Loan_Status']= y_predict['Loan_Status'].map({1: 'Y', 0: 'N'})
        y_predict.to_csv("Submission.csv")
        print(y_predict.head())

In [17]:
pred_obj = Prediction()
pred_obj.prediction(test_data)

Gender               367
Married              367
Dependents           367
Education            367
Self_Employed        367
ApplicantIncome        0
CoapplicantIncome      0
LoanAmount             0
Loan_Amount_Term       6
Credit_History         0
Property_Area        367
dtype: int64


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').